In [1]:
clusters = {
  "cl1": {
    "wn1": {
      "pod1": {
        "c1": {
          "name": "01_cl1_wn1_pod1_c1",
          "path": "/home/gull-shair-butt/Documents/Pycharm-Projects/FYP/admin/pet_store/new_configs/01_cl1_wn1_pod1_c1.json",
          "tag": "01",
          "image": "segullshairbutt/website:01_cl1_wn1_pod1_c1_image"
        },
        "c2": {
          "name": "01_cl1_wn1_pod1_c2",
          "path": "/home/gull-shair-butt/Documents/Pycharm-Projects/FYP/admin/pet_store/new_configs/01_cl1_wn1_pod1_c2.json",
          "tag": "01",
          "image": "segullshairbutt/website:01_cl1_wn1_pod1_c2_image"
        },
        "c3": {
          "name": "01_cl1_wn1_pod1_c3",
          "path": "/home/gull-shair-butt/Documents/Pycharm-Projects/FYP/admin/pet_store/new_configs/01_cl1_wn1_pod1_c3.json",
          "tag": "01",
          "image": "segullshairbutt/website:01_cl1_wn1_pod1_c3_image"
        },
        "c4": {
          "name": "01_cl1_wn1_pod1_c4",
          "path": "/home/gull-shair-butt/Documents/Pycharm-Projects/FYP/admin/pet_store/new_configs/01_cl1_wn1_pod1_c4.json",
          "tag": "01",
          "image": "segullshairbutt/website:01_cl1_wn1_pod1_c4_image"
        }
      }
    },
    "wn2": {
      "pod1": {
        "c1": {
          "name": "01_cl1_wn2_pod1_c1",
          "path": "/home/gull-shair-butt/Documents/Pycharm-Projects/FYP/admin/pet_store/new_configs/01_cl1_wn2_pod1_c1.json",
          "tag": "01",
          "image": "segullshairbutt/website:01_cl1_wn2_pod1_c1_image"
        }
      }
    }
  }
}

In [70]:
def get_template(containers_config, wn_name, count):
    tag = containers_config[0]['tag']
    
    containers = ""
    for container_config in containers_config:
        containers += "\n"
        containers += get_containers(container_config['name'], container_config['image'])        
    
    return f"""---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: config-{tag}-{count}-deployment
  labels: 
    app: config-{tag}-{count}
spec:
  replicas: 1
  selector:
    matchLabels:
      app: config-{tag}-{count}
  template:
    metadata:
      labels:
        app: config-{tag}-{count}
    spec:
      containers:
      {containers}
      nodeSelector: 
        name: {wn_name}
---
apiVersion: v1
kind: Service
metadata:
  name: config-{tag}-{count}-service
spec: 
  selector:
    app: config-{tag}-{count}
  ports:
    - protocol: TCP
      port: 27017
      targetPort: 27017"""


def get_containers(name, image_name):
    return f"""      - name: {name}
        image: {image_name}"""

In [71]:
templates = ""
for cl_name, cluster in clusters.items():
    count = 0
    for wn_name, pods in cluster.items():
        count += 1
        for pod_name, containers in pods.items():
            containers = list(containers.values())
            if containers:
                templates = templates + get_template(containers, wn_name, count) + "\n"

In [72]:
import yaml
with open('output.yaml', 'w') as output_file:
    output_file.write(templates)
